In [1]:
# !jupyter nbextension enable --py widgetsnbextension --sys-prefix
# !jupyter serverextension enable voila --sys-prefix

In [2]:
import cv2

In [3]:
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box, Text

In [4]:
from IPython.display import display, clear_output

In [5]:
from lib.image import image_data
from lib.analysis import dyssynchrony_analysis

In [6]:
import numpy as np
import pandas as pd

In [17]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings("ignore", category=RuntimeWarning) 

In [7]:
def cardiaAp_analyzer(image):
    # Select ROI
    fromCenter = False
    showCrosshair = False


    #seleted_parameters = cv2.selectROI(image, fromCenter, showCrosshair)
    
    # Crop image
    #x_start = int(seleted_parameters[1])
    #x_end = x_start + int(seleted_parameters[3])
    #y_start = int(seleted_parameters[0]) 
    #y_end = y_start + int(seleted_parameters[2])

    #cv2.destroyAllWindows()
    #image = image_data.get_image_data(image)
    #image = image_data.crop_vertical(image, x_start, x_end)
    #image = image_data.crop_horizontal(image, y_start, y_end)

    return dyssynchrony_analysis.analyze_image(image, \
                                               min_dist_between_maxs.value, \
                                               calibration.value, \
                                               slice_width=slice_width.value)

In [8]:
#Uploader button
uploader = widgets.FileUpload(accept='.tif', multiple=False)

#Input text for settings
slice_width = widgets.FloatText(
    value=0,
    description='Slice width:',
    disabled=False    
)

slice_width = widgets.IntText(
    value=0,
    description='Slice width:',
    disabled=False
)


min_dist_between_maxs = widgets.IntText(
    value=0,
    description='Dist between maxs:',
    disabled=False
)

calibration = widgets.IntText(
    value=0,
    description='Calibration:',
    disabled=False
)

In [9]:
html = "<h1 style='font-size:25px; color:black; margin-left: 15px'> Welcome to Cardi<b>AP</b> </h1>"
header_text = widgets.HTML(value=html)

In [10]:
html_text_upload = "<p style='font-size:15px; color:black'> Upload you image to inicialize the analysis</p>"
header_text_upload = widgets.HTML(value=html_text_upload)

box_upload = Layout(display='flex',
                    flex_flow='flex-wrap',
                    width='50%',
                    margin='0px 10px 10px 0px',
                    padding='0px 25px 25px 25px',
                    justify_content='space-between')

vbox_upload = widgets.VBox([header_text_upload, uploader], layout=box_upload)

In [11]:
#Display the div
box_layout = Layout(display='flex',
                    flex_flow='column',
                    align_items='stretch',
                    width='50%',
                    margin='0px 10px 10px 0px',
                    padding='0px 25px 25px 25px',
                    justify_content='space-between')

vbox_settings = widgets.VBox([slice_width, min_dist_between_maxs, calibration], layout=box_layout)

In [12]:
analyze_btn = widgets.Button(description="Analyze", button_style='danger')

def load_cv2_image_from_bytes(bytes_):
    return cv2.imdecode(np.frombuffer(bytes_, dtype=np.uint8), cv2.IMREAD_COLOR)

def get_uploader_content(uploader):
    return uploader.value[list(uploader.value)[0]]["content"]

results_box = widgets.Output()

@results_box.capture()
def on_button_clicked(_button):
    print("Analyzing...")
    global analysis_results
    results_box.clear_output()
    analysis_results = cardiaAp_analyzer(load_cv2_image_from_bytes(get_uploader_content(uploader)))
    display(pd.DataFrame(analysis_results["slices"]))
    clear_uploader(uploader)
    
        
analyze_btn.on_click(on_button_clicked)

In [13]:
def clear_uploader(uploader):
    uploader.value.clear()
    uploader._counter = 0

In [18]:
clear_uploader(uploader)
display(header_text)
display(vbox_upload)
display(vbox_settings)

display(analyze_btn)
display(results_box)

HTML(value="<h1 style='font-size:25px; color:black; margin-left: 15px'> Welcome to Cardi<b>AP</b> </h1>")

Button(button_style='danger', description='Analyze', style=ButtonStyle())

Output(outputs=({'output_type': 'stream', 'text': '/home/ana/Documents/CardiAp/lib/analysis/statistical_functi…